<a href="https://colab.research.google.com/github/VictorFu0717/AI/blob/main/sentiment_gap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 4s 0us/step


In [ ]:
import glob
import os
dn = os.path.dirname(dataset)
fn = glob.glob(dn + "/aclImdb/train/pos/*")
with open(fn[0], "r", encoding="utf-8") as f:
    print("pos:", f.read())
fn = glob.glob(dn + "/aclImdb/train/neg/*")
with open(fn[0], "r", encoding="utf-8") as f:
    print("neg:", f.read())

pos: Kudos to Cesar Montano for reviving the Cebuano movie! Panaghoy sa Suba is very good -- it has the drama, the action, the romance, and scene that will make you laugh.<br /><br />While the story is not that original (a love triangle -- or make a four-cornered-love, Japanese occupation, rebellion, American as lord), its presentation is something cool, especially it uses it original language -- bisaya for the Filipino, nipongo for the Japanese and English for the American.<br /><br />This movie will go as one of this year's best Pinoy movies.<br /><br />Go watch this!
neg: Highly implausible, unbelievable, and incoherent Spanish production about...well, let me see just how close I can get to it. The film opens with a woman having one of her cat's killed by a young girl. She then begs her lover to take her somewhere on his vacation. He calls work and demands that he loses his vacation time and she says he will pay for this. What relevance this plays out to is anybody's guess at the fi

In [ ]:
import pandas as pd
def getdata(base):
    datas = {"article":[], "ans":[]}
    targets = os.path.join(base, "pos", "*")
    for fn in glob.glob(targets):
        with open(fn, "r", encoding="utf-8") as f:
            datas["article"].append(f.read())
            datas["ans"].append(1)
    targets = os.path.join(base, "neg", "*")
    for fn in glob.glob(targets):
        with open(fn, "r", encoding="utf-8") as f:
            datas["article"].append(f.read())
            datas["ans"].append(0)
    return pd.DataFrame(datas)
train = os.path.join(dn, "aclImdb", "train")
train_df = getdata(train)
test = os.path.join(dn, "aclImdb", "test")
test_df = getdata(test)

In [ ]:
test_df

,article,ans
0,Excellent Hitchcock thriller with Robert Cummi...,1
1,Having read the book prior to watching this ad...,1
2,NO SPOILERS!!<br /><br />After Hitchcock's suc...,1
3,Another Raquel Welch Classic! This Picture hit...,1
4,Contains Spoilers<br /><br />This is a Peter W...,1
...,...,...
24995,Angels and Demons: 3 out of 10: Clearly someth...,0
24996,This odd little film starts out with the story...,0
24997,I challenge you to watch this film and deny th...,0
24998,"A confusing, senseless script with plot holes ...",0


In [ ]:
#num_words=3000最常出現的3000個單字才做處理，其他的忽略
#token將詞轉成數字 Tokenize: I -> 1 love -> 2 you -> 3
#texts to sequences將整句文字轉成數字列

from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["article"])
x_train_seq = tok.texts_to_sequences(train_df["article"])
x_test_seq = tok.texts_to_sequences(test_df["article"])

In [ ]:
# tok.word_index, 0不會用到, 0 for padding
tok.index_word[34]
# 有東西消失是因為超過三千最常出現單字
tok.sequences_to_texts(x_train_seq)[0]

'to for the movie is very good it has the drama the action the romance and scene that will make you laugh br br while the story is not that original a love or make a four love japanese american as lord its presentation is something cool especially it uses it original language for the for the japanese and english for the american br br this movie will go as one of this best movies br br go watch this'

In [ ]:
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,5,15,1,17,6,52,49,9,44.0,1.0,450.0,1.0,202.0,1.0,880.0,2.0,133.0,12.0,77.0,94.0,22.0,459.0,7.0,7.0,134.0,1.0,62.0,6.0,21.0,12.0,201.0,3.0,116.0,39.0,94.0,3.0,685.0,116.0,857.0,295.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,17,13,138,3,9,44,12,231.0,49.0,243.0,481.0,8.0,126.0,390.0,2161.0,12.0,69.0,5.0,3.0,17.0,9.0,44.0,3.0,49.0,746.0,1069.0,247.0,1065.0,56.0,735.0,3.0,144.0,289.0,243.0,56.0,149.0,337.0,1581.0,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2990,15,175,144,448,88,1326,29,893.0,34.0,2078.0,53.0,8.0,1.0,519.0,2.0,286.0,4.0,1.0,1155.0,153.0,520.0,1478.0,51.0,33.0,119.0,15.0,1.0,120.0,8.0,1220.0,2.0,742.0,196.0,1127.0,14.0,734.0,14.0,9.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,5,2537,1656,15,51,211,47,26.0,655.0,43.0,12.0,44.0,38.0,212.0,604.0,1.0,248.0,171.0,1022.0,1.0,238.0,1092.0,1624.0,67.0,229.0,26.0,490.0,5.0,26.0,67.0,40.0,184.0,266.0,5.0,165.0,15.0,297.0,1965.0,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,17,163,3,2576,41,2306,1255,48.0,26.0,15.0,2.0,48.0,1.0,1409.0,2247.0,16.0,29.0,1.0,2024.0,1773.0,4.0,3.0,810.0,81.0,25.0,1467.0,1.0,1409.0,16.0,1.0,1409.0,2.0,1670.0,6.0,21.0,16.0,1.0,1409.0,243.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,8,1,703,4,3,2852,31,2228,6.0,2405.0,20.0,83.0,2622.0,33.0,42.0,12.0,294.0,14.0,3.0,198.0,4.0,10.0,67.0,401.0,1.0,49.0,99.0,20.0,28.0,505.0,21.0,770.0,58.0,369.0,20.0,1.0,1880.0,17.0,1.0,1214.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,47,68,3,173,4,368,84,186,99.0,1052.0,8.0,1.0,18.0,11.0,19.0,431.0,215.0,28.0,4.0,95.0,42.0,3.0,899.0,1.0,503.0,215.0,125.0,14.0,9.0,492.0,36.0,3.0,539.0,323.0,12.0,301.0,8.0,3.0,375.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,634,27,1199,1,279,10,1607,11,17.0,13.0,85.0,10.0,241.0,3.0,663.0,334.0,4.0,88.0,36.0,399.0,234.0,26.0,629.0,296.0,1.0,49.0,230.0,10.0,470.0,5.0,64.0,87.0,14.0,8.0,3.0,272.0,214.0,43.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,329,20,2,190,851,22,97,604,232.0,4.0,126.0,110.0,13.0,12.0,29.0,7.0,7.0,11.0,19.0,728.0,43.0,5.0,2238.0,143.0,3.0,679.0,389.0,510.0,295.0,981.0,8.0,1.0,402.0,45.0,994.0,337.0,2.0,424.0,994.0,204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 個人覺得512 1024差不多
MAXLEN = 512

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=MAXLEN)
x_test_pad = pad_sequences(x_test_seq, maxlen=MAXLEN)

In [ ]:
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,14,1631,91,2974,6,139,642,258,9,1074,9,201,1098,15,1,15,1,857,2,628,15,1,295,7,7,11,17,77,137,14,28,4,11,115,99,7,7,137,103,11
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,69,5,3,17,9,44,3,49,746,1069,247,1065,56,735,3,144,289,243,56,149,337,1581,29,278,4,250,10,383,11,17,15,256,34,735,46,482,18,6,79,3
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,73,312,1,120,2401,234,10,470,5,839,36,29,286,1301,29,8,29,1,120,13,3,144,1689,15,69,204,293,9,20,285,447,208,2,25,506,9,254,55,7,7
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6,28,4,1,830,1624,8,1,476,4,1624,155,43,4,155,45,22,25,32,815,11,497,42,251,5,166,340,85,54,2058,11,893,15,454,30,1,2016,3,400,155
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,272,2,800,671,6,260,746,2,48,72,755,15,11,17,5,1972,12,18,47,6,35,73,4,110,12,188,27,2373,8,3,2684,238,531,17,11,13,3,63,84,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,23,57,2597,160,39,57,1,219,224,1094,9,301,46,429,4,84,20,1,19,170,5,21,57,76,28,459,43,4,69,7,7,58,1239,1206,7,7,741,1916,268,365
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,14,1,1977,165,704,2,29,1,153,23,1917,222,63,21,73,10,67,132,41,1,19,441,14,47,215,73,5,9,1,503,6,3,186,507,2,10,431,89,383,9
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,588,13,181,1136,14,1,2308,844,90,2343,303,14,865,14,56,97,76,56,767,69,1,88,2,13,1301,1075,14,32,5,1,220,45,61,1,226,119,1348,5,1,153
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,45,61,5,2282,16,1,2595,18,47,215,98,57,1,1377,6,614,30,1,127,4,3,2585,264,1040,2,7,7,14,22,235,479,1,19,44,3,651,274,48,3,2141


In [ ]:
#Embedding詞嵌入((3000詞＋1padding),128種情緒）
#mask_zero：True, 0不列入運算，視為padding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D
layers = [
    Embedding(3001, 128, mask_zero=True, input_length=MAXLEN),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 不需要自己做one-hot
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [ ]:
import numpy as np
y_train = np.array(train_df["ans"])
y_test = np.array(test_df["ans"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
   EarlyStopping(patience=5, restore_best_weights=True),
   ModelCheckpoint("sentiment.h5", save_best_only=True)
]
model.fit(x_train_pad,
     y_train,
     batch_size=200,
     epochs=50,
     validation_split=0.1,
     verbose=2,
     callbacks=callbacks)

Epoch 1/50
113/113 - 7s - loss: 0.6348 - accuracy: 0.6428 - val_loss: 0.6364 - val_accuracy: 0.6424
Epoch 2/50
113/113 - 3s - loss: 0.4672 - accuracy: 0.8268 - val_loss: 0.4854 - val_accuracy: 0.7760
Epoch 3/50
113/113 - 3s - loss: 0.3616 - accuracy: 0.8671 - val_loss: 0.4033 - val_accuracy: 0.8168
Epoch 4/50
113/113 - 3s - loss: 0.3114 - accuracy: 0.8837 - val_loss: 0.3806 - val_accuracy: 0.8240
Epoch 5/50
113/113 - 3s - loss: 0.2825 - accuracy: 0.8923 - val_loss: 0.3741 - val_accuracy: 0.8316
Epoch 6/50
113/113 - 3s - loss: 0.2632 - accuracy: 0.8996 - val_loss: 0.3642 - val_accuracy: 0.8376
Epoch 7/50
113/113 - 3s - loss: 0.2493 - accuracy: 0.9048 - val_loss: 0.3163 - val_accuracy: 0.8660
Epoch 8/50
113/113 - 3s - loss: 0.2383 - accuracy: 0.9092 - val_loss: 0.3363 - val_accuracy: 0.8564
Epoch 9/50
113/113 - 3s - loss: 0.2301 - accuracy: 0.9114 - val_loss: 0.3205 - val_accuracy: 0.8668
Epoch 10/50
113/113 - 3s - loss: 0.2234 - accuracy: 0.9148 - val_loss: 0.3177 - val_accuracy: 0.8700

In [ ]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 3ms/step - loss: 0.2954 - accuracy: 0.8792


[0.2954466640949249, 0.8791599869728088]

In [ ]:
#將128種情緒提取出來

In [ ]:
layers = [
    Embedding(3001, 128, mask_zero=True),
    GlobalAveragePooling1D()
]
temp = Sequential(layers)
temp.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         384128    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [ ]:

w = model.layers[0].get_weights()
temp.layers[0].set_weights(w)

In [ ]:
# 用夾角來看相似度,0度：像, 180度：不像, 取cos來表示（1:像,0:不相干,-1不像）

In [ ]:
from numpy import dot
from numpy.linalg import norm

w1 = tok.word_index["father"]
pre = temp.predict([[w1]])
e1 = pre[0]
w2 = tok.word_index["dad"]
pre = temp.predict([[w2]])
e2 = pre[0]
print(w1, w2)
cos_sim = dot(e1, e2)/(norm(e1)*norm(e2))
print("相似度", cos_sim)

333 1243
相似度 -0.51157343


In [ ]:
s1 = "this movie is interesting"
w1 = [tok.word_index[w] for w in s1.split(" ")]
pre = temp.predict([w1])
e1 = pre[0]
s2 = "this movie is boring"
w2 = [tok.word_index[w] for w in s2.split(" ")]
pre = temp.predict([w2])
e2 = pre[0]
print(w1, w2)
cos_sim = dot(e1, e2)/(norm(e1)*norm(e2))
print("相似度", cos_sim)

[11, 17, 6, 218] [11, 17, 6, 355]
相似度 0.9844145
